In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim


In [ ]:
from neuio.tokenizer import Tokenizer
from neuio.activation import ReLU
from neuio.vectorizer import BuildVectors
from neuio.linear import Linear
class Softmax:
    def __init__(self, dim=1):
        self.dim = dim

    def forward(self, x):
        exp_vals = np.exp(x - np.max(x, axis=self.dim, keepdims=True))
        self.output = exp_vals / np.sum(exp_vals, axis=self.dim, keepdims=True)
        return self.output

    def backward(self, grad_output):
        softmax_output = self.output
        grad_input = grad_output * softmax_output * (1 - softmax_output)
        return grad_input



In [47]:
import numpy as np
class ReLU(nn.Module):
    def __init__(self, inplace=False):
        super(ReLU, self).__init__()
        self.inplace = inplace
    def _max(self,x):
        try:
            return np.maximum(0,x)
        except:
            return 0
    def forward(self, x):
        return self._max(x)

class Loss:
    @staticmethod
    def mean_squared_error(y_true, y_pred):
        return np.mean((y_true - y_pred)**2)
    @staticmethod
    def binary_cross_entropy(y_true, y_pred):
        epsilon = 1e-7   
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)   
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    @staticmethod
    def categorical_cross_entropy(y_true, y_pred):
        epsilon = 1e-7   
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)   
        return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

In [ ]:
df = pd.read_csv('./twitter_training.csv')
df = df.dropna()
text_data = df['text']
sentiment_data = df['sentiment']
label_encoder = Tokenizer()
sentiment_labels = label_encoder.fit_tokenize(sentiment_data)



In [53]:

vectorizer = BuildVectors()
train_vectors = vectorizer._fit_vectorize(text_data)


In [100]:
import numpy as np

class SentimentClassifier:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden = Linear(input_size, hidden_size)
        self.relu = ReLU()
        self.output = Linear(hidden_size, output_size)
        self.softmax = Softmax(dim=1)
        self.loss = Loss.categorical_cross_entropy

    def forward(self, x):
        hidden = self.relu(self.hidden.forward(x))
        output = self.softmax.forward(self.output.forward(hidden))
        return output

    def train(self, x, y, learning_rate=0.1, num_epochs=10):
        for epoch in range(num_epochs):
            print(f"Epoch: {epoch}")
            output = self.forward(x)
            loss = self.loss(output, y)
            grad_output = self.softmax.backward(output - y)
            grad_hidden = self.output.backward(grad_output, learning_rate)
            grad_input = self.hidden.backward(grad_hidden, learning_rate)
            
            print(f"Loss: {loss}")
    def predict(self, x):
        output = self.forward(x)
        return np.argmax(output, axis=1)

def shape(array):
    if isinstance(array, list):
        return (len(array),) + shape(array[0])
    else:
        return ()


In [101]:
input_size = shape(train_vectors)[1]
hidden_size = 64
output_size = len(label_encoder.classes)  

In [102]:
model = SentimentClassifier(input_size, hidden_size, output_size)



In [ ]:
model.train(train_vectors, sentiment_labels)


In [ ]:
def predict_text(model, text):
    text_vector = vectorizer.transform([text])
    
    model.predict(text_vector)
    # Get the predicted sentiment label
    _, predicted_index = torch.max(output, 1)
    predicted_label = label_encoder.inverse_transform(predicted_index.numpy())[0]

    return predicted_label



In [ ]:

# Test the predict_text function
text = "how are you"
predicted_sentiment = predict_text(model, text)
print(f"Predicted sentiment: {predicted_sentiment}")

In [ ]:
import numpy as np

def custom_matmul(input1, input2):
    # Get the dimensions of the input matrices
    m1, n1 = input1.shape
    m2, n2 = input2.shape

    # Check if the matrices can be multiplied
    if n1 != m2:
        raise ValueError("Invalid matrix dimensions for multiplication")

    # Create an empty result matrix
    result = np.zeros((m1, n2))

    # Perform matrix multiplication
    for i in range(m1):
        for j in range(n2):
            for k in range(n1):
                result[i, j] += input1[i, k] * input2[k, j]

    return result

# Example usage
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
result = custom_matmul(A, B)
print(result)


In [ ]:
torch.tensor(0)